# Create the .h5 dataset from .db annotations and .fasta sequences

In [ ]:
import json

import numpy as np
import pandas as pd
import h5py

import sqlite3
from pyfaidx import Faidx

from pwas.shared_utils.util import start_log, log

ANNOTATION_COUNTS_CSV_FILE_PATH = '/cs/phd/nadavb/cafa_project/data/unique_annotations_counts.csv'
ANNOTATIONS_SQLITE_FILE_PATH = '/cs/phd/nadavb/cafa_project/data/protein_annotations.db'
FASTA_FILE_PATH = '/cs/phd/nadavb/cafa_project/data/uniref90.fasta'

OUTPUT_H5_FILE_PATH = '/cs/phd/nadavb/cafa_project/data/dataset.h5'

start_log('/cs/phd/nadavb/logs', 'cafa_create_h5_dataset')

def format_quantity(quantity):
    if quantity is None:
        return 'all'
    else:
        return str(quantity)

def load_seqs_and_annotations(annotations_limit = None, shuffle = False, print_progress_interval = 10000):

    log('Loading %s GO annotations...' % format_quantity(annotations_limit))
    cnx = sqlite3.connect(ANNOTATIONS_SQLITE_FILE_PATH)
    raw_go_annotations = pd.read_sql_query('SELECT * FROM protein_annotations' + ('' if annotations_limit is None else \
            (' LIMIT %d' % annotations_limit)), cnx)
    log('Loaded %d GO annotations (%d columns: %s)' % (raw_go_annotations.shape + (', '.join(raw_go_annotations.columns),)))
    
    if shuffle:
        raw_go_annotations = raw_go_annotations.sample(frac = 1, random_state = 0)

    log('Loading Faidx (%s)...' % FASTA_FILE_PATH)
    seqs_faidx = Faidx(FASTA_FILE_PATH)
    log('Finished loading Faidx.')
    
    n_failed = 0

    for i, (uniprot_id, raw_annotations) in raw_go_annotations[['uniprot_name', 'complete_go_annotation_indices']].iterrows():
        
        if i % print_progress_interval == 0:
            log('%d/%d' % (i, len(raw_go_annotations)), end = '\r')
        
        seq_fasta_id = 'UniRef90_%s' % uniprot_id.split('_')[0]
        
        try:
            seq = str(seqs_faidx.fetch(seq_fasta_id, 1, seqs_faidx.index[seq_fasta_id].rlen))
            yield uniprot_id, seq, json.loads(raw_annotations)
        except KeyError:
            n_failed += 1
            
    log('Finished. %d of %d sequences failed.' % (n_failed, len(raw_go_annotations)))

annotation_counts = pd.read_csv(ANNOTATION_COUNTS_CSV_FILE_PATH, index_col = 0, squeeze = True, names = ['count'])
common_annotations = np.array(sorted(annotation_counts[annotation_counts >= 100].index))
common_annotation_to_index = {annotation: i for i, annotation in enumerate(common_annotations)}
n_common_annotations = len(common_annotations)
log('Will encode the %d most common annotations.' % n_common_annotations)

def encode_annotations(annotations):
    
    annotation_mask = np.zeros(n_common_annotations, dtype = bool)
    
    for annotation in annotations:
        if annotation in common_annotation_to_index:
            annotation_mask[common_annotation_to_index[annotation]] = True
            
    return annotation_mask

n_seqs = sum(1 for _ in load_seqs_and_annotations())
log('Will create a dataset of %d final sequences.' % n_seqs)

with h5py.File(OUTPUT_H5_FILE_PATH, 'w') as h5f:
    
    h5f.create_dataset('included_annotation_indices', data = common_annotations, dtype = np.int32)
    uniprot_ids = h5f.create_dataset('uniprot_ids', shape = (n_seqs,), dtype = h5py.string_dtype())
    seqs = h5f.create_dataset('seqs', shape = (n_seqs,), dtype = h5py.string_dtype())
    seq_lengths = h5f.create_dataset('seq_lengths', shape = (n_seqs,), dtype = np.int32)
    annotation_masks = h5f.create_dataset('annotation_masks', shape = (n_seqs, n_common_annotations), dtype = bool)
    
    for i, (uniprot_id, seq, annotations) in enumerate(load_seqs_and_annotations(shuffle = True)):
        uniprot_ids[i] = uniprot_id
        seqs[i] = seq
        seq_lengths[i] = len(seq)
        annotation_masks[i, :] = encode_annotations(annotations)
                
log('Done.')

Creating log file: cafa_create_h5_dataset__28708__2020_06_04-11:14:45.txt
[2020_06_04-11:14:45] Will encode the 8943 most common annotations.
[2020_06_04-11:14:45] Loading all GO annotations...
[2020_06_04-11:20:21] Loaded 106555993 GO annotations (8 columns: index, tax_id, uniprot_name, go_annotations, flat_go_annotations, n_go_annotations, complete_go_annotation_indices, n_complete_go_annotations)
[2020_06_04-11:20:21] Loading Faidx (/cs/phd/nadavb/cafa_project/data/uniref90.fasta)...
[2020_06_04-11:24:47] Finished loading Faidx.
[2020_06_04-17:22:44] Finished. 349008 of 106555993 sequences failed.
[2020_06_04-17:40:16] Will create a dataset of 106206985 final sequences.
[2020_06_04-17:40:18] Loading all GO annotations...
[2020_06_04-17:46:15] Loaded 106555993 GO annotations (8 columns: index, tax_id, uniprot_name, go_annotations, flat_go_annotations, n_go_annotations, complete_go_annotation_indices, n_complete_go_annotations)
[2020_06_04-17:48:16] Loading Faidx (/cs/phd/nadavb/cafa_

# Shuffle the dataset

Apparently it is not that useful (creating the dataset from scratch appears to be faster)

In [ ]:
import numpy as np
import pandas as pd
import h5py

from pwas.shared_utils.util import start_log, log, get_chunk_intervals

INPUT_H5_FILE_PATH = '/cs/phd/nadavb/cafa_project/data/dataset.h5'
OUTPUT_SHUFFLED_H5_FILE_PATH = '/cs/phd/nadavb/cafa_project/data/shuffled_dataset.h5'

CHUNK_SIZE = 100000

start_log('/cs/phd/nadavb/logs', 'cafa_create_h5_dataset')

def slice_h5_data_in_arbitrary_order(data, indices):
    '''
    This is required because h5py forces you to slice in increasing order.
    '''
    sorted_indices = pd.Series(np.array(indices)).sort_values()
    sliced_data_in_increasing_order = data[sorted_indices.values]
    sliced_data_in_desired_order = np.empty(sliced_data_in_increasing_order.shape, dtype = data.dtype)
    sliced_data_in_desired_order[sorted_indices.index] = sliced_data_in_increasing_order
    return sliced_data_in_desired_order

with h5py.File(INPUT_H5_FILE_PATH, 'r') as h5f_input, h5py.File(OUTPUT_SHUFFLED_H5_FILE_PATH, 'w') as h5f_output:
    
    included_annotation_indices = h5f_input['included_annotation_indices'][:]
    n_seqs = len(h5f_input['seq_lengths'])
    
    h5f_output.create_dataset('included_annotation_indices', data = included_annotation_indices)
    seq_lengths = h5f_output.create_dataset('seq_lengths', shape = (n_seqs,), dtype = np.int32)
    seqs = h5f_output.create_dataset('seqs', shape = (n_seqs,), dtype = h5py.string_dtype())
    annotation_masks = h5f_output.create_dataset('annotation_masks', shape = (n_seqs, len(included_annotation_indices)), dtype = bool)
    
    shuffled_indices = np.arange(n_seqs)
    np.random.seed(0)
    np.random.shuffle(shuffled_indices)
    
    for i1, i2 in get_chunk_intervals(n_seqs, CHUNK_SIZE):
        
        log('Setting %d..%d/%d...' % (i1, i2, n_seqs))
        chunk_indices = shuffled_indices[i1:i2]
        
        seq_lengths[i1:i2] = slice_h5_data_in_arbitrary_order(h5f_input['seq_lengths'], chunk_indices)
        seqs[i1:i2] = slice_h5_data_in_arbitrary_order(h5f_input['seqs'], chunk_indices)
        annotation_masks[i1:i2] = slice_h5_data_in_arbitrary_order(h5f_input['annotation_masks'], chunk_indices)

log('Done.')

[2020_06_03-14:38:56] Setting 0..100000/106206985...
[2020_06_03-15:32:49] Setting 100000..200000/106206985...
[2020_06_03-16:52:42] Setting 200000..300000/106206985...
[2020_06_03-18:10:26] Setting 300000..400000/106206985...
[2020_06_03-19:28:26] Setting 400000..500000/106206985...
[2020_06_03-20:44:16] Setting 500000..600000/106206985...
[2020_06_03-21:59:43] Setting 600000..700000/106206985...
